In [ ]:
라이브러리 임포트

In [2]:
import os, cv2, numpy as np
import imutils, time
from urllib import request # 다운로드를 위한 임포트

def download(url , model): # 모델 다운 및 저장 함수
    print('download', model)
    file = request.urlopen(url+model).read()
    open(model, "wb").write(file)

In [ ]:
Python과 OpenCV를 사용한 신경 스타일 전이

In [6]:
# 사전학습 모델 사이트 및 모델 이름
path = 'http://cs.stanford.edu/people/jcjohns/fast-neural-style/models/eccv16/'
model = 'starry_night.t7'
# model = 'the_wave.t7' # 다른 사전학습 모델 다운 가능
# model = 'la_muse.t7'
# model = ' composition_vii.t7’

# 사전학습 모델 없으면 다운로드
print(os.path.exists(model))
if os.path.exists(model) == False : download(path , model)
print("loading style transfer model...")
net = cv2.dnn.readNetFromTorch(model)

True
loading style transfer model...


In [7]:
image = cv2.imread('../images/monalisa.jpg') # 콘텐트 영상 읽기
print(image.shape)
image = imutils.resize(image, width=600) # 너비 600으로 크기 변경
(h, w) = image.shape[:2]
b, g, r, _ = cv2.mean(image) # 채널별 평균 값 계산

# 영상에서 blob을 만들어 입력으로 설정한 다음 네트워크의 순방향 수행
blob = cv2.dnn.blobFromImage(image, 1.0, (w, h), (b, g, r), swapRB=False, crop=False)
net.setInput(blob)
start = time.time() # 수행시간 측정
output = net.forward() # 순방향 전파
end = time.time()

# 출력 텐서를 다시 형성하고 평균 뺄셈 복귀
output = output.reshape((3, output.shape[2], output.shape[3]))
output[0] += b
output[1] += g
output[2] += r

# 형태 변경: 채널 × 높이 × 너비 → 세로 × 높이 × 채널
output = output.transpose(1, 2, 0)
print("neural style transfer took {:.4f} seconds".format(end - start))
# 결과 영상 표시 및 저장
# cv2.imshow("output", cv2.convertScaleAbs(output, 255))
# cv2.waitKey()
cv2.imwrite('output/styled.jpg', output) # 결과 영상 저장

(340, 228, 3)
neural style transfer took 1.5870 seconds


True